# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [ ]:
!pip install openai

In [2]:

from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [3]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [ ]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {
    'table': ['users', 'orders', 'products', 'payments'],  # Replace with your column names
    'definition': [
        'Information about the users of the platform',
        'Details of customer orders including date and total amount',
        'List of products available for sale',
        'Records of payments made by customers'
    ]  # Replace with your definitions
}

df = pd.DataFrame(data)
print(df)

In [5]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [ ]:
print(text_tables)

In [7]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [8]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question= "What is the purpose of this table?")

In [ ]:
print(return_OAI(pqt1))

In [10]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="What is the purpose of this table?")

In [ ]:
print(return_OAI(pqt3))

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [ ]:
!pip install openai==0.28

In [13]:
# Import necessary libraries
import pandas as pd
import openai
import os
from dotenv import load_dotenv, find_dotenv

# Load API key from .env file
_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Set the OpenAI API key
openai.api_key = OPENAI_API_KEY

# Define the table data and create a DataFrame
data = {
    'table': ['users', 'orders', 'products', 'payments'],
    'definition': [
        'Information about the users of the platform',
        'Details of customer orders including date and total amount',
        'List of products available for sale',
        'Records of payments made by customers'
    ]
}
df = pd.DataFrame(data)


In [14]:

# Format table definitions into a string
text_tables = "\n".join([f"{row['table']}: {row['definition']}" for _, row in df.iterrows()])


In [15]:

# Define prompt templates
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Question:
{question}
"""


In [16]:

# Define a function to query GPT
def query_gpt(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{'role': 'system', 'content': prompt}],
        temperature=0
    )
    return response.choices[0].message['content']

# Experiment with different questions
questions = [
    "What are the total sales by product category?",
    "How many users made purchases last month?",
    "Which products generated the most revenue?",
    "What is the average payment amount recorded?"
]


In [ ]:

# Generate and test prompts
results = []
for question in questions:
    
    prompt = prompt_question_tables.format(tables=text_tables, question=question)
    print(f"Prompt:\n{prompt}\n")

    response = query_gpt(prompt)
    print(f"Response:\n{response}\n")
    results.append({'question': question, 'response': response})


In [18]:

# Create a report summarizing findings
report = """
# Findings Report on GPT Prompt Responses

## Prompts Tested:
{}
""".format("\n".join([f"- {q['question']}" for q in results]))

for i, result in enumerate(results):
    report += f"""
## Question {i + 1}: {result['question']}
### Prompt:
{prompt_question_tables.format(tables=text_tables, question=result['question'])}
### Response:
{result['response']}
"""


In [ ]:

# Save the report to a file
report_file = "gpt_responses_report.txt"
with open(report_file, "w") as file:
    file.write(report)

print(f"Report saved to {report_file}.")
